In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scripts import  CompanyStockPriceScraper
from scripts import ADXIndex
from scipy.signal import find_peaks

## Select interesting examples

Interesting examples are those where: 
- there is a peak in the ADX index close to the day of publication,
- there is a low ADX index value over an extended period before and after the publication date. 

In [2]:
# Analysis company list 
tickers = ["CCC","CDL","EAH","HRP","IPO","LBW","LPP","MIR","MON","PRT","SFG","SNW","WTN"] # WIG_ODZIEZ
ticker = "PRT"

In [3]:
# Scrap stock value and calculate ADX index
csp = CompanyStockPriceScraper()
adx = ADXIndex()
stock_value = csp.getHistoricalPrice(ticker)
adx_index = adx.cast_to_data_frame(adx.get_index(stock_value))
stock_value = pd.concat([stock_value, adx_index], axis=1, join="inner")

In [4]:
# Get date of report publication
report_database = pd.read_csv("../database/mergedData/Annual_V.csv", index_col=[0])
report_days = report_database[report_database["Ticker"] == ticker]["Data"].to_numpy()

In [5]:
# Find inner index day
inner_index_day = np.array([np.sum(stock_value.index < report_day) for report_day in report_days])
inner_index_day = inner_index_day[inner_index_day != 0]

In [6]:
STRONG_TRENDS_RANGE = 10 # +/- days from publication date
DETECT_TRENDS_VALUE = 40 # Value ADX bigger than interpret as strong trends

In [7]:
# Check is in report publication day surroundings strong trends
# Strong trends ADX > 40 if ADX periods = 14
peak_threshold = stock_value["ADX"].mean() + stock_value["ADX"].std()

for report_day_idx in inner_index_day:
    if any(stock_value[report_day_idx-STRONG_TRENDS_RANGE:
                       report_day_idx+STRONG_TRENDS_RANGE+1]["ADX"] > DETECT_TRENDS_VALUE):
        
        # Find distance date report - ADX peak 
        peaks_idx, _ = find_peaks(stock_value["ADX"], height=DETECT_TRENDS_VALUE, distance=STRONG_TRENDS_RANGE)
        distance = peaks_idx[np.abs(report_day_idx - peaks_idx).argmin()] - report_day_idx
        print("Important")
        print(distance)
        print("-------")
    else:
        print("Not Important")
        print("-------")
        

Important
1
-------
Not Important
-------
Important
6
-------
Not Important
-------
Important
-13
-------
Important
4
-------
Not Important
-------
Not Important
-------
Important
-21
-------
Not Important
-------
Not Important
-------
Not Important
-------
Important
-11
-------
Important
6
-------
Important
-1
-------
Not Important
-------
Not Important
-------
Not Important
-------
Not Important
-------


In [8]:
adx.data_frame_to_dict(stock_value)

{'ADX': 2000-12-29    28.035001
 2001-01-01    28.035001
 2001-01-02    29.475921
 2001-01-03    30.196382
 2001-01-04    31.510715
                 ...    
 2024-09-27    29.832362
 2024-09-30    29.708389
 2024-10-01    31.131382
 2024-10-02    32.452732
 2024-10-03    33.679700
 Name: ADX, Length: 6123, dtype: float64,
 '+DI': 2000-12-29    35.982499
 2001-01-01    35.982499
 2001-01-02    33.821119
 2001-01-03    33.821119
 2001-01-04    31.615976
                 ...    
 2024-09-27    13.931563
 2024-09-30    13.252041
 2024-10-01    10.951095
 2024-10-02    10.233523
 2024-10-03     9.629513
 Name: +DI, Length: 6123, dtype: float64,
 '-DI': 2000-12-29    64.017501
 2001-01-01    64.017501
 2001-01-02    66.178881
 2001-01-03    66.178881
 2001-01-04    68.384024
                 ...    
 2024-09-27    24.819290
 2024-09-30    23.608711
 2024-10-01    32.531761
 2024-10-02    30.400113
 2024-10-03    28.605815
 Name: -DI, Length: 6123, dtype: float64,
 'Indexes': DatetimeIndex(['